# MusicBrainz artist lookup

Documentation:
 - Web service: https://wiki.musicbrainz.org/Development/XML_Web_Service/Version_2/Search
 - Artist entities: https://musicbrainz.org/doc/Artist
 - Area entities: https://musicbrainz.org/doc/Area

Sample queries:
 - Artist: https://musicbrainz.org/ws/2/artist/d4659efb-b8eb-4f03-95e9-f69ce35967a9
 - Area: https://musicbrainz.org/ws/2/area/0a70f24b-1263-4341-8d70-17b8df84154f?inc=area-rels

In [ ]:
import pandas as pd, requests, time, json

pause_standard = 0.1
pause_exceeded_rate = 19

In [ ]:
# configure URLs and user-agent header
artist_name_url = u'https://musicbrainz.org/ws/2/artist/?query=artist:{}&fmt=json'
artist_id_url = u'https://musicbrainz.org/ws/2/artist/{}?fmt=json'
area_id_url = u'https://musicbrainz.org/ws/2/area/{}?inc=area-rels&fmt=json'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

## Define functions

In [ ]:
def make_request(url, headers=headers):
    
    global pause_standard
    global pause_exceeded_rate
    
    time.sleep(pause_standard)
    #print url
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200: #if status OK
        return {'status_code':response.status_code, 'json':response.json()}
    
    elif response.status_code == 503: #if status error (server busy or rate limit exceeded)
        if 'exceeding the allowable rate limit' in response.json()['error']:
            #print '\n{}\n{}\npausing {} secs and re-try\n'.format(response.status_code, response.json(), pause_exceeded_rate)
            pause_standard = pause_standard + 0.1
            print 'pause_standard={}'.format(pause_standard),
            time.sleep(pause_exceeded_rate)
        return make_request(url)
    
    else: #if other status code, print info and return None for caller to handle
        print 'make_request error code {} {}'.format(response.status_code, response.json())
        return None

In [ ]:
def geocode(address):
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = u'http://maps.googleapis.com/maps/api/geocode/json?address={}&sensor=false' #api url with placeholders
    request = url.format(address) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = json.loads(response.text) #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract lat-long from result
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return '{},{}'.format(latitude, longitude)

In [ ]:
def get_artist_id_by_name(name):
    try:
        response = make_request(artist_name_url.format(name))
        if response is not None:
            result = response['json']
            artist_id = result['artists'][0]['id']
            return artist_id
    except:
        print 'get_artist_id_by_name error: {}'.format(response)

In [ ]:
def get_artist_by_id(artist_id):
    try:
        response = make_request(artist_id_url.format(artist_id))
        if response is not None:
            result = response['json']
            data = {'id':artist_id,
                    'name':result['name'],            
                    'type':result['type'],
                    'gender':result['gender'],
                    'country':result['country'],
                    'begin_date':None,
                    'end_date':None,
                    'area_id':None,
                    'area_name':None,
                    'area_name_full':None,
                    'area_latlng':None,
                    'begin_area_id':None,
                    'begin_area_name':None,
                    'begin_area_name_full':None,
                    'begin_area_latlng':None,
                    'place':None,
                    'place_id':None,
                    'place_full':None,
                    'place_latlng':None}

            if result['life-span'] is not None and 'begin' in result['life-span'] and 'end' in result['life-span']:
                data['begin_date'] = result['life-span']['begin']
                data['end_date'] = result['life-span']['end']
            if result['area'] is not None and 'id' in result['area'] and 'name' in result['area']:
                data['area_id'] = result['area']['id']
                data['area_name'] = result['area']['name']
            if result['begin_area'] is not None and 'id' in result['begin_area'] and 'name' in result['begin_area']:
                data['begin_area_id'] = result['begin_area']['id']
                data['begin_area_name'] = result['begin_area']['name']
            
            # populate place with begin_area_name if it's not null, else area_name if it's not null, else None
            if data['begin_area_name'] is not None:
                data['place'] = data['begin_area_name']
                data['place_id'] = data['begin_area_id']
            elif data['area_name'] is not None:
                data['place'] = data['area_name']
                data['place_id'] = data['area_id']
            
            return data
    
    except:
        print 'get_artist_by_id error: {}'.format(response)

In [ ]:
def get_area(area_id, area_str=''):
    try: 
        response = make_request(area_id_url.format(area_id))
        if response is not None:
            result = response['json']
            
            if area_str == '':
                area_str = result['name']
            
            if 'relations' in result:
                for relation in result['relations']:
                    if relation['direction']=='backward' and relation['type']=='part of':
                        area_id = relation['area']['id']
                        area_str = u'{}, {}'.format(area_str, relation['area']['name'])
                        return area_id, area_str
            else:
                print 'get_area no relations error: {}'.format(result)
            return None, area_str
    except:
        print 'get_area error: {}'.format(response)

In [ ]:
def get_place_full_name_by_id(area_id):
    area_name=''
    while area_id is not None:
        area_id, area_name = get_area(area_id, area_name)
    return area_name

In [ ]:
def make_artists_df(artist_ids, row_labels=None, df=None, status_interval=10):
    
    # create a list of row labels if caller didn't pass one in
    if row_labels is None:
        row_labels = range(len(artist_ids))
    
    # create a new dataframe if caller didn't pass an existing one in
    if not isinstance(df, pd.DataFrame):
        cols = ['id', 'name', 'type', 'gender', 'country', 'begin_date', 'end_date', 'begin_area_name', 
                'begin_area_name_full', 'begin_area_latlng', 'area_name', 'area_name_full', 'area_latlng',
                'place_id', 'place', 'place_full', 'place_latlng']
        df = pd.DataFrame(columns=cols)
    
    start_time = time.time()
    for artist_id, n in zip(artist_ids, row_labels):

        if n % status_interval == 0 :
            print n,
            df.to_csv('data/mb.csv', index=False, encoding='utf-8')

        try:
            # get the artist info object
            artist = get_artist_by_id(artist_id)

            # create (or update) a df row containing the data from this artist object
            df.loc[n] = [ artist[col] for col in cols ]

            # update the row's place_full cell with the full place name
            df.loc[n, 'place_full'] = get_place_full_name_by_id(artist['place_id'])

        except:
            print '#{} failed'.format(n)
            pass

    df.to_csv('data/mb.csv', index=False, encoding='utf-8')
    finish_time = time.time()
    print '\nprocessed {:,} artists in {:,} seconds'.format(len(artist_ids), round(finish_time-start_time, 2))
    
    return df

## Test it with a demo

In [ ]:
name = 'david bowie'
artist_id = get_artist_id_by_name(name)
artist = get_artist_by_id(artist_id)
artist['place_full'] = get_place_full_name_by_id(artist['place_id'])
artist['place_full']

## Now run it

In [ ]:
scrobbles = pd.read_csv('data/lastfm_scrobbles.csv', encoding='utf-8')
artist_ids = scrobbles['artist_mbid'].dropna().unique()
len(artist_ids)

In [ ]:
df = make_artists_df(artist_ids)

In [ ]:
df.head()

## Re-try any failed rows one more time

In [ ]:
# first get all the rows missing place_full (ie, row was created but couldn't get full place name)
rows_missing_place_full = list(df[pd.isnull(df['place_full'])].index)

# then get all the row labels missing in the df (due to errors that prevented row creation)
missing_row_labels = [ label for label in range(len(artist_ids)) if label not in df.index ]

# combine the two lists then get the artist mbid for each
row_labels_to_retry = sorted(rows_missing_place_full + missing_row_labels)
artist_ids_to_retry = [ artist_ids[index] for index in row_labels_to_retry ]

print '{} artists to retry'.format(len(artist_ids_to_retry))

In [ ]:
df = make_artists_df(artist_ids_to_retry, row_labels_to_retry, df, status_interval=1)

In [ ]:
df.head()

In [ ]:
# OK, one final check - see how many artist ids did not make it into the final dataframe
# first get all the rows missing place_full (ie, row was created but couldn't get full place name)
rows_missing_place_full = list(df[pd.isnull(df['place_full'])].index)

# then get all the row labels missing in the df (due to errors that prevented row creation)
missing_row_labels = [ label for label in range(len(artist_ids)) if label not in df.index ]

print len(rows_missing_place_full)
print len(missing_row_labels)